In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from colorcloud.biasutti2019riu import LitModel, LitDataModule
import lightning as L
import wandb
from lightning.pytorch.loggers import WandbLogger
from datetime import datetime


In [ ]:
model = LitModel()
data = LitDataModule(train_batch_size=10)

In [ ]:
time = datetime.now()
timestamp = str(time.year) + '-' + str(time.month) + '-' + str(time.day) + '_' + str(time.hour) + '-' + str(time.minute) + '-' + str(time.second)

In [ ]:
wandb_logger = WandbLogger(project="colorcloud", name=timestamp, log_model="all")
wandb_logger.watch(model.net, log="all")

In [ ]:
# train model
trainer = L.Trainer(max_epochs=2, logger=wandb_logger)
trainer.fit(model, data)

In [ ]:
wandb.finish()